In [19]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [20]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    # r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    r'D:\code\forecast_model\notebook\20250828_tcn_data',
    ]

In [21]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250828_data_P100_bc_420_seed0_IDX0_final,20250828_data_P100_bc_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499272,99.253665,40.872588,55.632824,48.863974,0.03h,5.97h,0.741045,0.777031,0.703338
1,20250828_data_P100_bc_420_seed0_IDX0_best,20250828_data_P100_bc_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499272,99.253665,40.872588,55.632824,48.863974,0.03h,5.97h,0.747575,0.777031,0.703338
2,20250828_data_P100_bc_420_seed1_IDX2_final,20250828_data_P100_bc_420_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.501937,98.172216,38.031681,43.999898,44.873419,0.03h,6.31h,0.728759,0.722788,0.692832
3,20250828_data_P100_bc_420_seed1_IDX2_best,20250828_data_P100_bc_420_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.501937,98.172216,38.031681,43.999898,44.873419,0.03h,6.31h,0.748459,0.722788,0.692832
4,20250828_data_P100_bc_420_seed2_IDX4_final,20250828_data_P100_bc_420_seed2_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.500070,98.935943,36.605217,41.749954,44.407890,0.03h,6.04h,0.729337,0.708849,0.683121
5,20250828_data_P100_bc_420_seed2_IDX4_best,20250828_data_P100_bc_420_seed2_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.500070,98.935943,36.605217,41.749954,44.407890,0.03h,6.04h,0.688813,0.708849,0.683121
6,20250828_data_P100_bc_420_seed3_IDX6_final,20250828_data_P100_bc_420_seed3_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.498081,99.746662,41.136271,53.243831,48.744141,0.03h,6.54h,0.740907,0.763279,0.702974
7,20250828_data_P100_bc_420_seed3_IDX6_best,20250828_data_P100_bc_420_seed3_IDX6,train_data,no,200,128,8192,0.0384,0,0,...,0.498081,99.746662,41.136271,53.243831,48.744141,0.03h,6.54h,0.735356,0.763279,0.702974
8,20250828_data_P100_bc_420_seed4_IDX8_final,20250828_data_P100_bc_420_seed4_IDX8,train_data,no,200,128,8192,0.0384,0,0,...,0.498134,99.521024,39.577001,54.655557,47.041404,0.04h,7.21h,0.722428,0.770393,0.695281
9,20250828_data_P100_bc_420_seed4_IDX8_best,20250828_data_P100_bc_420_seed4_IDX8,train_data,no,200,128,8192,0.0384,0,0,...,0.498134,99.521024,39.577001,54.655557,47.041404,0.04h,7.21h,0.774582,0.770393,0.695281


In [22]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [23]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [24]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250828_data_P100_bc_420_seed0_IDX0_final,1(309275)@1(309275)
1,20250828_data_P100_bc_420_seed0_IDX0_best,1(309275)@1(309275)
2,20250828_data_P100_bc_420_seed1_IDX2_final,1(309275)@1(309275)
3,20250828_data_P100_bc_420_seed1_IDX2_best,1(309275)@1(309275)
4,20250828_data_P100_bc_420_seed2_IDX4_final,1(309275)@1(309275)
5,20250828_data_P100_bc_420_seed2_IDX4_best,1(309275)@1(309275)
6,20250828_data_P100_bc_420_seed3_IDX6_final,1(309275)@1(309275)
7,20250828_data_P100_bc_420_seed3_IDX6_best,1(309275)@1(309275)
8,20250828_data_P100_bc_420_seed4_IDX8_final,1(309275)@1(309275)
9,20250828_data_P100_bc_420_seed4_IDX8_best,1(309275)@1(309275)


In [ ]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_loss']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))

# # 只考察 final
# _match_bool = data['train_title'].apply(lambda x: 'final' in x)
# d = d.loc[_match_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values('val_f1',ascending=False)


,train_loss,train_f1,val_f1,cost
train_title,,,,
20250828_data_P100_bc_420_final,0.015150,0.994624,0.695981,6.414h
20250828_data_P100_bc_420_best,0.015150,0.994624,0.695981,6.414h
20250828_data_P100_deeplob_420_final,0.014125,0.994961,0.681926,6.244h
20250828_data_P100_deeplob_420_best,0.014125,0.994961,0.681926,6.244h
20250828_data_P100_TimeSeriesStaticModelx8_deeplob_300,0.013003,0.995435,0.680547,3.04h
20250828_data_P100_TimeSeriesStaticModelx8_bc_300,0.012255,0.995675,0.676436,3.06h
20250828_data_P100_TimeSeriesStaticModelx8_bc_200,0.009046,0.996900,0.668479,3.074h
20250828_data_P100_TimeSeriesStaticModelx8_deeplob_200,0.008475,0.997078,0.655824,3.068h


In [26]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]